<a href="https://colab.research.google.com/github/leohcar/P_IA025A_2022S1/blob/main/entrega_final/Proyecto_encoder_resnet50_smalldataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder

In [1]:
nome = 'Carlos Leonardo Ancasi Hinostroza'

print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


# Importação das bibliotecas

In [2]:
import collections
import itertools
import functools
import math
import random

import torch
import torchvision
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from torchvision.datasets import MNIST

In [3]:
# Check which GPU we are using
!nvidia-smi

Sat Jul 16 01:46:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print('GPU available:', device)

GPU available: cuda


In [5]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

#Dataset e dataloader

In [6]:
!mkdir ./data

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize([224,224]),
#      torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform = torchvision.transforms.Compose(
     [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
print(f'numero de imagenes {len(train_dataset)}')
img, target = train_dataset[0]
print(img.size())

numero de imagenes 50000
torch.Size([3, 32, 32])


In [8]:
# train_size = 40000
# val_size = 10000
# train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
train_size = 10
val_size = 10000
res_size = 50000 - train_size - val_size
train_dataset, val_dataset, _ = torch.utils.data.random_split(train_dataset, [train_size, val_size, res_size])

In [9]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 1
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([10, 3, 32, 32])
Valores mínimo e máximo dos pixels:  tensor(-1.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


#Bloco Básico da Resnet

In [10]:
from collections import OrderedDict
from torch import nn
from torch import Tensor


class MyBlock(nn.Module):

    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:
        super(MyBlock, self).__init__()
        # Escreva seu código aqui.
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # Escreva seu código aqui.
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [11]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:

        super(Bottleneck,self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=1, bias=False, padding=0)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.conv3 = nn.Conv2d(planes, planes*self.expansion, kernel_size=1, stride=1, bias=False, padding=0)
        self.bn3 = nn.BatchNorm2d(planes*self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)

        out += identity
        out = self.relu(out)
        return out


In [12]:
class CNN_resnet50(nn.Module):
    def __init__(self, image_channels, num_classes):

        super(CNN_resnet50,self).__init__()

        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #resnet layers
        self.layer1 = self.__make_layer(64, 64, 3,stride=1)
        self.layer2 = self.__make_layer(256, 128, 4,stride=2)
        self.layer3 = self.__make_layer(512, 256, 6,stride=2)
        self.layer4 = self.__make_layer(1024, 512, 3,stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.fc = nn.Linear(2048, num_classes)
      
    def __make_layer(self, in_channels, out_channels, blocks, stride):

        identity_downsample = None
        
        if stride != 1 or in_channels != 4*out_channels:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels, 4*out_channels, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(4*out_channels))
        layers = []
        layers.append(Bottleneck(in_channels, out_channels,identity_downsample=identity_downsample, stride=stride))
        
        for _ in range(1,blocks):
            layers.append(Bottleneck(out_channels*4, out_channels))

        layers.append(nn.Dropout(0.3))

        return nn.Sequential(*layers)
    
    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(x.shape[0], -1)
        out = self.fc(out)
        return out


In [13]:
model = CNN_resnet50(3,100).to(device)

In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 23712932


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
model_save_name = 'model_resnet50_smalldataset.pt'
path_modelo = F"/content/drive/My Drive/modelo_proyecto/entrega_03/pre_trained/{model_save_name}"

In [17]:
# Iniciar o treinamento
from tqdm import tqdm

n_epochs = 80
lr = 3.3e-4

img_channels = 3
num_classes = 100

model = CNN_resnet50(img_channels,num_classes).to(device)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

checkpoint = {
    'img_channels': img_channels,
    'num_classes': num_classes,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'epochs': [],
    'loss_train': [],
    'loss_val': [],
    'accuracy_history': [],
    }
torch.save(checkpoint, path_modelo)

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = []
accuracy_history = []
total_trained_samples = 0

with torch.no_grad():
    loss_accuracy = [list(validation_step(val_input_ids.to(device), val_target_ids.to(device)))
        for val_input_ids, val_target_ids in val_dataloader]
    
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()
    print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
    accuracy_history.append(mean_accuracy.item())

    checkpoint['accuracy_history'].append(mean_accuracy.item())
    checkpoint['loss_val'].append(mean_loss.item())
    torch.save(checkpoint, path_modelo)
    
for i in range(n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())
    
    checkpoint['model_state_dict'] = model.state_dict()
    checkpoint['optimizer_state_dict'] = optimizer.state_dict()

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0114 Loss_Val: 4.605016374588013


Training: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


Epoch: 0/79 Loss_Train: 5.7896728515625
Accuracy: 0.009399999999999999 Loss_Val: 4.605762302875519


Training: 100%|██████████| 1/1 [00:00<00:00, 10.58it/s]


Epoch: 1/79 Loss_Train: 4.8218488693237305
Accuracy: 0.009399999999999999 Loss_Val: 4.6077459836006165


...

Epoch: 77/79 Loss_Train: 0.013261361047625542
Accuracy: 0.008400000000000001 Loss_Val: 14.699252896308899


Training: 100%|██████████| 1/1 [00:00<00:00, 14.71it/s]


Epoch: 78/79 Loss_Train: 0.0035925847478210926
Accuracy: 0.0078000000000000005 Loss_Val: 14.576575326919556


Training: 100%|██████████| 1/1 [00:00<00:00, 11.34it/s]


Epoch: 79/79 Loss_Train: 0.000456485606264323
Accuracy: 0.0076 Loss_Val: 14.517796478271485


In [18]:
# continuar com o  treino
from tqdm import tqdm

n_epochs = 160
lr = 1e-4

checkpoint = torch.load(path_modelo)

model = CNN_resnet50(checkpoint['img_channels'],checkpoint['num_classes']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = checkpoint['loss_train']
accuracy_history = checkpoint['accuracy_history']
total_trained_samples = 0
epochs_init = checkpoint['epochs'][-1]+1

for s in zip(checkpoint['accuracy_history'],checkpoint['loss_val'],checkpoint['epochs'],checkpoint['loss_train']):
    print(f'Accuracy: {s[0]} Loss_Val: {s[1]}')
    print(f'')
    print(f'Epoch: {s[2]:d}/{n_epochs - 1:d} Loss_Train: {s[3]}')
print(f"Accuracy: { checkpoint['accuracy_history'][-1] } Loss_Val: {checkpoint['loss_val'][-1]}")

    
for i in range(epochs_init,n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())

    checkpoint['model_state_dict'] = model.state_dict()
    checkpoint['optimizer_state_dict'] = optimizer.state_dict()

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0114 Loss_Val: 4.605016374588013

Epoch: 0/159 Loss_Train: 5.7896728515625
Accuracy: 0.009399999999999999 Loss_Val: 4.605762302875519

Epoch: 1/159 Loss_Train: 4.8218488693237305
Accuracy: 0.009399999999999999 Loss_Val: 4.6077459836006165

Epoch: 2/159 Loss_Train: 4.162418365478516
Accuracy: 0.0123 Loss_Val: 4.611880030632019

Epoch: 3/159 Loss_Train: 3.4088878631591797
Accuracy: 0.0123 Loss_Val: 4.61944442987442

Epoch: 4/159 Loss_Train: 2.953382968902588
Accuracy: 0.0104 Loss_Val: 4.632846174240112

Epoch: 5/159 Loss_Train: 2.0935957431793213
Accuracy: 0.0104 Loss_Val: 4.655354082584381

Epoch: 6/159 Loss_Train: 1.8244295120239258
Accuracy: 0.0104 Loss_Val: 4.689003872871399

Epoch: 7/159 Loss_Train: 2.123354434967041
Accuracy: 0.011000000000000001 Loss_Val: 4.732852113246918

Epoch: 8/159 Loss_Train: 2.6274619102478027
Accuracy: 0.011000000000000001 Loss_Val: 4.788303771018982

Epoch: 9/159 Loss_Train: 1.8189306259155273
Accuracy: 0.011000000000000001 Loss_Val: 4.8613545

Training: 100%|██████████| 1/1 [00:00<00:00,  4.22it/s]


Epoch: 80/159 Loss_Train: 0.0021986456122249365
Accuracy: 0.0076 Loss_Val: 14.542079763412476


Training: 100%|██████████| 1/1 [00:00<00:00, 13.55it/s]


Epoch: 81/159 Loss_Train: 0.00430176081135869
Accuracy: 0.0074 Loss_Val: 14.32928611755371


...

Epoch: 157/159 Loss_Train: 0.00044985272688791156
Accuracy: 0.0077 Loss_Val: 14.420218486785888


Training: 100%|██████████| 1/1 [00:00<00:00, 14.52it/s]


Epoch: 158/159 Loss_Train: 0.001488102599978447
Accuracy: 0.0078000000000000005 Loss_Val: 14.480855741500854


Training: 100%|██████████| 1/1 [00:00<00:00, 14.00it/s]


Epoch: 159/159 Loss_Train: 0.00015721135423518717
Accuracy: 0.0077 Loss_Val: 14.487541255950928


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=batch_size)

with torch.no_grad():
    loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
        for val_input_ids, val_target_ids in val_dataloader]
        
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()

print(f'TEST: Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')